# Visualising results of GAIL hyperparmeter sweep

In [ ]:
%matplotlib inline

In [ ]:
import cloudpickle
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from skopt.plots import plot_evaluations, plot_objective

In [ ]:
sns.set(context='notebook', style='darkgrid')

In [ ]:
pickle_path = glob.glob('../ray-tune-results/ray_tune_trial/search-alg-*.pkl')[-1]
with open(pickle_path, 'rb') as fp:
    _, skopt_alg = cloudpickle.load(fp)
skopt_res = skopt_alg.get_result()
dim_names = [
    'omit_noop', 'disc_up_per_iter', 'disc_replay_mult', 'disc_lr', 'disc_use_act',
    'disc_all_frames', 'sampler_time_steps', 'sampler_batch_envs', 'ppo_lr',
    'ppo_gamma', 'ppo_lambda', 'ppo_ent', 'ppo_adv_clip', 'ppo_norm_adv'
]

In [ ]:
_ = plot_evaluations(skopt_res)  # , dimensions=dim_names)

In [ ]:
_ = plot_objective(skopt_res, n_samples=40) # , minimum='expected_minimum_random', n_minimum_search=1000)  # , dimensions=dim_names)

In [ ]:
n_results = len(skopt_res.func_vals)
sns.distplot(skopt_res.func_vals, rug=True, norm_hist=False, kde=False, bins=10 if n_results >= 20 else None)
plt.title(f"Negative final score distribution from {n_results} runs (lower = better)")
plt.xlabel("Negative final score")
plt.ylabel("Frequency")
plt.show()

In [ ]:
import skopt
import matplotlib
import jupyterlab
print('skopt:', skopt.__version__)
print('seaborn:', sns.__version__)
print('matplotlib:', matplotlib.__version__)
print('jupyterlab:', jupyterlab.__version__)

In [ ]:
from skopt.utils import expected_minimum_random_sampling
skopt_min = expected_minimum_random_sampling(skopt_res, n_random_starts=100000)
print("minimum from random sampling:", skopt_min)

In [ ]:
good_inds, = np.nonzero(skopt_res.func_vals <= -0.95)
for conf_num, good_ind in enumerate(good_inds, start=1):
    print(f"Good config at index {good_ind} ({conf_num}/{len(good_inds)}):")
    for dimension, value in zip(skopt_res.space.dimensions, skopt_res.x_iters[good_ind]):
        print(f'    {dimension.name} = {value}')
        
print('Amalgamated "good" configs:')
for dim_idx, dimension in enumerate(skopt_res.space.dimensions):
    values = [skopt_res.x_iters[i][dim_idx] for i in good_inds]
    if isinstance(values[0], float):
        values_str = f"[{', '.join('%.3g' % v for v in values)}]"
    else:
        values_str = str(values)
    print(f'    {dimension.name} = {values_str}')